In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time
from scipy.signal import convolve2d
from ConvLayerPara import ConvLayerP

In [2]:
batch = 2
C, M, H, W, K = 3, 3, 32, 32, 5

x_shape = (batch, C,H,W)
m_shape = (M,C,K,K)
y_shape = (batch, M,H-K+1,W-K+1)

X = np.random.rand(*x_shape).astype(np.float32)
print('Shape of X: ', X.shape)
Masks = np.random.rand(*m_shape).astype(np.float32)
print('Shape of Masks: ', Masks.shape)

Shape of X:  (2, 3, 32, 32)
Shape of Masks:  (3, 3, 5, 5)


In [4]:
layer = ConvLayerP()

Y_naive = layer.forward_naive(X, Masks, batch, C, M, H, W, K)
print(Y_naive.shape)
# print(Y_naive)

(32, 32, 1)
(1000, 16, 1)
(1000, 16, 28, 28)


In [3]:
layer = ConvLayerP()

Y_shared = layer.forward_shared(X, Masks, batch, C, M, H, W, K)
print(Y_shared.shape)
print(Y_shared)



Block:  (32, 32, 1)
Grid:  (2, 3, 1)
sharedmem:  5284 Byte
(2, 3, 28, 28)
[[[[  0.         0.         0.       ... 227.69893  227.69893
    227.69893 ]
   [227.69893  227.69893  227.69893  ...  87.820625  65.035
     65.035   ]
   [ 65.035     65.035     36.442947 ...   0.         0.
      0.      ]
   ...
   [  0.         0.         0.       ...   0.         0.
      0.      ]
   [  0.         0.         0.       ...   0.         0.
      0.      ]
   [  0.         0.         0.       ...   0.         0.
      0.      ]]

  [[  0.         0.         0.       ...   0.         0.
      0.      ]
   [  0.         0.         0.       ...   0.         0.
      0.      ]
   [  0.         0.         0.       ...   0.         0.
      0.      ]
   ...
   [  0.         0.         0.       ...   0.         0.
      0.      ]
   [  0.         0.         0.       ...   0.         0.
      0.      ]
   [  0.         0.         0.       ...   0.         0.
      0.      ]]

  [[  0.         0.     

In [8]:
np.dtype(np.float32).itemsize

4